# Workshop 4

Starter code for workshop 4. You should have seen most of it before, but make sure you understand what it is doing!

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To plot even prettier figures
import seaborn as sn

# General data handling (pure numerics are better in numpy)
import pandas as pd

In [ ]:
# We can import the dataset in this way
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [ ]:
xarray = data.data      # Extract the features
yarray = data.target    # Extract the target
print(xarray.shape)
print(yarray.shape)
# We can concatenate features with target
fullarray = np.concatenate((xarray,np.reshape(yarray,(-1,1))),axis=1)
print(fullarray.shape)

In [ ]:
print(f'Features names are: {data.feature_names}')
print(f'Label names are: {data.target_names}')
# We want to consider malignant as the 'positive' class 
#  - otherwise interpretation gets harder as a positive test in medicine corresponds to more severe disease

In [ ]:
# This point is pretty important
# For general convenience, class 1 is the positive class and in general the positive class is the worst condition

# Now invert the labels (so that malignant=1)
fullarray[:,-1] = 1 - fullarray[:,-1]   
df = pd.DataFrame(fullarray,columns = list(data.feature_names) + ['target'])

In [ ]:
df.describe()

In [ ]:
## Note that the mean of the target column gives the percentage of samples with target=1 (i.e. 37% has target=1, and so 63% have target=0)
## If there were more classes it would be important to determine and display the relevant proportions for all classes in another way

In [ ]:
print(df.count())

In [ ]:
print(np.sum(df.isna()))

In [ ]:
dummy = df.hist(bins=40,figsize=(20,15))

In [ ]:
plt.figure(figsize=(20,22))
for n in range(fullarray.shape[1]):
    plt.subplot(6,6,n+1)
    plt.plot(np.sort(fullarray[:,n]),'o')
    plt.title(df.columns[n])
plt.show()

# Splitting into separate datasets

In [ ]:
from sklearn.model_selection import train_test_split

bigtrain_set, test_set = train_test_split(fullarray, test_size=0.2, random_state=42)
train_set, val_set = train_test_split(bigtrain_set, test_size=0.2, random_state=42)

In [ ]:
# Get x and y for train, val and test
X_train = train_set[:,:-1]
y_train = train_set[:,-1]
X_test = test_set[:,:-1]
y_test = test_set[:,-1]
X_val = val_set[:,:-1]
y_val = val_set[:,-1]
print([X_train.shape,y_train.shape,X_test.shape,y_test.shape,X_val.shape,y_val.shape])

# Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Replace missing features with median, and scale to std distribution
preproc_pl = Pipeline([('imputer', SimpleImputer(strategy="median")), 
                          ('stdscaler', StandardScaler())])

# SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

# This is a logistic regression
sgd_pl = Pipeline([ ('preproc',preproc_pl), ('sgd',SGDClassifier(loss='log')) ])
sgd_pl.fit(X_train,y_train)
y_val_pred = sgd_pl.predict(X_val)
y_val_prob = sgd_pl.predict_proba(X_val)

In [ ]:
# Check what is the result in y_val_pred and y_val_prob

In [ ]:
y_val_prob = y_val_prob[:,1]   # just take the positive class probabilities

In [ ]:
# This plot can help you  to find the errors
plt.plot(np.abs(y_val - y_val_pred),'r*')
plt.xlabel('Sample Number')
plt.ylabel('Error Status')
plt.title('Errors - Binary')
plt.show()
plt.plot(np.abs(y_val - y_val_prob),'b*')
plt.xlabel('Sample Number')
plt.ylabel('Error Status')
plt.title('Errors - Probability Difference')
plt.show()

In [ ]:
# Why in the second plot I have a similar result than the first plot?

# Check the values again, but using two decimals only
np.round(y_val_prob,2)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cmat = confusion_matrix(y_true=y_val, y_pred=y_val_pred)
sn.heatmap(cmat,annot=True)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# positive = worse/most impotant condition = 1
# negative = less important condition = 0 

# Can you give me?

# accuracy = 
# true_positive (TP) = 
# true_negative (TN) = 
# false_positive (FP) = 
# false_negative (FN) = 

In [ ]:
# Using normalize = true, you get the proportion of cases related to the rows
cmat = confusion_matrix(y_true=y_val, y_pred=y_val_pred, normalize='true')
sn.heatmap(cmat,annot=True)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Normalised Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
# The most common metric
sgd_acc = accuracy_score(y_true = y_val, y_pred = y_val_pred) # -> right predictions / total of samples
print(f'Accuracy for SGD is {sgd_acc}')

In [ ]:
from sklearn.metrics import precision_score, recall_score
# Other useful metrics
prec = precision_score(y_true = y_val, y_pred = y_val_pred) # -> TP / (TP + FP) 
recl = recall_score(y_true = y_val, y_pred = y_val_pred)    # -> TP / (TP + FN)
print(f'Precision & Recall are {prec} and {recl}')

* **Precision** gives relevance to the FP cases
* **Recall** gives relevance to the FN cases

Which one is more important? it depends

**Example 1**: You are creating a model to predict the condition of tumors (benign and malignant). What is the most important error?

In this context:

* False negative (FN) is when you are predicting the tumor is benign, but actually is malignant
* False positive (FP) is when you are predicting the tumor is malignant, but actually is benign

FN cases could be much worse, so, for this case is more important avoid FN -- > recall

**Example 2**: You are creating a pregancy test (using ML) to predict whether a woman is pregnant or not. What is the most important error?

In this context:

* False negative (FN) is when you are predicting the woman is not pregnant, but actually she is pregnant.
* False positive (FP) is when you are predicting the woman is pregnant, but actually is not pregnant.

In **my personal opinion**, a FP case is worse, so, you should try to avoid these cases --> precision.

### **Receiver-Operator-Characteristic (ROC)**

It illustrates the diagnostic ability of a binary classifier considering different **thresholds**.
It compare the performance of:


* **TPR** (recall): TP / (TP + FN) --> **How good is my model in predicting correctly the positive class?**
* **FPR**: FP / (FP + TN) --> **how often the model makes false positive prediction?**

In [ ]:
from sklearn.metrics import roc_curve, auc

# Original curve
fpr, tpr, thresholds = roc_curve(y_val, y_val_prob, pos_label=1)
plt.plot(fpr,tpr,'b')
plt.xlabel('FPR')
plt.ylabel('TPR')

### EXTENSION CODE ###

# Fake a change in one point
ridx = np.random.randint(y_val_prob.shape[0])
y_val_prob[ridx] = 1 - y_val_prob[ridx]

# Curve with the modification
fpr, tpr, thresholds = roc_curve(y_val, y_val_prob, pos_label=1)
plt.plot(fpr,tpr,'r')
plt.title('ROC curve')

### END EXTENSION CODE ###

plt.show()

In [ ]:
# Here you can see the thresholds used in that model
print(thresholds)

In [ ]:
# Here you can see the samples when your model fails.
print(y_val_prob[y_val_pred!=y_val])

**Question:** *How many distinct points are there?  Try calculating the ROC curve again using the probability outputs instead. Look at the thresholds and compare these to the predicted probability outputs from the classifier just at the points where the binary prediction is wrong.*

There are 8 thresholds, representing 2 end points and 6 intermediate points, although several are extremely close to 0 or 1.


In [ ]:
# AUC stands for Area Under the Curve
auc_sgd = auc(fpr,tpr)
print(f'AUC for SGD classifier is {auc_sgd}')

In [ ]:
from sklearn.metrics import precision_recall_curve

# This is another illustration to evaluate the performance of a model.
prec, recl, thresholds = precision_recall_curve(y_val, y_val_prob, pos_label=1)
plt.plot(recl,prec,'b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

# A higher threshold leads to higher precision, but low recall

In [ ]:
print(thresholds)
print((np.min(y_val_prob),np.max(y_val_prob)))

**Questions:** *If this classifier would be used to make decisions in the hospital, which threshold would you choose? Is precision more important or recall? Do you think this classifier is good enough or does it need more research?*

You should consider this and decide for yourself, but realise that a False Negative is more serious than a False Positive, because if someone has a disease and is left untreated it could be very bad, whereas a healthy person who is told they have a disease will normally undergo more tests and then find out that it was a False Positive. Depending on the disease and the clinical pathway, there may be unnecessary treatment and stress involved for those that are in the False Positive group, but those that are in the False Negative group miss out on earlier treatment, and this can often have fatal consequences.

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

dt_pl = Pipeline([('preproc',preproc_pl), ('dt',DecisionTreeClassifier(random_state=0))])
dt_pl.fit(X_train,y_train)
y_val_pred_tree = dt_pl.predict(X_val)
y_val_prob_tree = dt_pl.predict_proba(X_val)

In [ ]:
plt.plot(np.abs(y_val - y_val_pred_tree),'r*')
plt.xlabel('Sample Number')
plt.ylabel('Error Status')
plt.title('Errors')
plt.show()

In [ ]:
# You can see the plot of your decision tree
plt.rcParams['figure.dpi'] = 200
dummy = plot_tree(dt_pl['dt'])

### **This plot gives you information about**

1) The column or feature used in each split.
2) **Gini**: The score given in each split using Gini impurity score
3) **samples**: Number of samples used in each split
4) **value**: How many samples I have for class 0 and class 1, respectively


**Question:** *What do each of the components (nodes, branches, thresholds) of the decision tree mean?*

You should be able to answer this on the basis of what was in the lecture. Note that feature numbers in each decision node are expressed as X[n]<=threshold

In [ ]:
plt.rcParams['figure.dpi'] = 80

In [ ]:
cmat = confusion_matrix(y_true=y_val, y_pred=y_val_pred_tree, normalize='true')
sn.heatmap(cmat,annot=True)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Normalised Confusion Matrix')
plt.show()

In [ ]:
dt_acc = accuracy_score(y_true = y_val, y_pred = y_val_pred_tree)
print(f'Accuracy for Decision Tree is {dt_acc}')

In [ ]:
prec = precision_score(y_true = y_val, y_pred = y_val_pred_tree)
recl = recall_score(y_true = y_val, y_pred = y_val_pred_tree)
print(f'Precision & Recall are {prec} and {recl}')

In [ ]:
fpr, tpr, thresholds = roc_curve(y_val, y_val_prob_tree[:,1], pos_label=1)
plt.plot(fpr,tpr,'b')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
print(thresholds)

**Questions:** *Why are there so few points in the ROC curve?  Does it still show useful information?*

You can see that there are only three points (two end points and one intermediate point) but it still indicates the single useful operating point and its associated performance values.

In [ ]:
auc_dt = auc(fpr,tpr)
print(f'AUC for Decision Tree classifier is {auc_dt}')

**Questions:** *How does the decision tree compare to the SGD linear model?  List 2 pros and 2 cons of each approach.*

AUC is 0.95 for the SGD linear model and 0.92 for the Decision Tree.  The confusion matrix tells a similar story.  Although SGD has better performance, the decision tree is more interpretable.  Can you think of other pros and cons?

In [ ]:
prec, recl, thresholds = precision_recall_curve(y_val, y_val_prob_tree[:,1], pos_label=1)
plt.plot(recl,prec,'b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

# Model selection

**Question:** *What do you think would be a good performance metric to use in this case, and why?  Choose one to work with here.*

A good answer here will depend on what you think is most important in the context of the task. If we want to try and supress False Negatives primarily then it would be good to choose an option with a good Recall, but still with acceptable Precision.  Based on the Precision-Recall curves, I would personally choose the SGD, with an operating point nearest to the top right. Looking at the class predictions (y_val_pred) as opposed to the probabilities (y_val_prob) shows that it is already choosing a good operating point, as shown also by the confusion matrices.

In [ ]:
print(f'Comparison based on AUC is {auc_sgd} vs {auc_dt} for SGD vs Decision Tree')

In [ ]:
print(f'Comparison based on accuracy is {sgd_acc} vs {dt_acc} for SGD vs Decision Tree')

In [ ]:
# Choosing SGD
bestmod = sgd_pl

In [ ]:
bestmod.fit(np.concatenate((X_train,X_val),axis=0),np.concatenate((y_train,y_val)))
y_test_pred = bestmod.predict(X_test)
y_test_prob = bestmod.predict_proba(X_test)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_test_prob[:,1], pos_label=1)
auc_bm = auc(fpr, tpr)
acc_bm = accuracy_score(y_test, y_test_pred)
print(f'Final scores on Test set are: AUC = {auc_bm} and Accuracy = {acc_bm}')
print(f'For comparison, scores on Validation set are: AUC = {auc_sgd} and Accuracy = {sgd_acc}')

**Question:** *What would it mean if there was a big difference between the performance scores on the validation and test datasets?*

These scores are quite similar and a big difference would not be expected in this case. The validation performance will typically be biased to be higher than it should, based on that fact that it was used to choose the best method, but for a small number of comparisons it would not normally be a large bias. If you do see a large difference then it might either be due to chance, or it may be due to a bug, so it is worth double-checking your code, looking more closely at the results and/or trying another random splitting.